In [1]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [154]:
import time

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torchvision import transforms
import matplotlib.pyplot as plt
from PIL import Image

In [161]:
class CNN(nn.Module):
  def __init__(self):
    super().__init__()  # Important, otherwise will throw an error

    self.c1 = torch.nn.Conv2d(3, 32, 5, padding=2)
    self.p1 = torch.nn.MaxPool2d(2)
    
    self.c2 = torch.nn.Conv2d(32, 64, 5, padding=2)
    self.p2 = torch.nn.MaxPool2d(2)

    self.c3 = torch.nn.Conv2d(64, 64, 5, padding=2)
    self.p3 = torch.nn.MaxPool2d(2)
    
    self.l1 = torch.nn.Linear(4 * 4 * 64, 1000)
    self.l2 = torch.nn.Linear(1000, 10)

  def forward(self, x):
    x = F.relu(self.c1(x))
    x = self.p1(x)
    
    x = F.relu(self.c2(x))
    x = self.p2(x)
        
    x = F.relu(self.c3(x))
    x = self.p3(x)
    
    x = torch.flatten(x, start_dim=1)
    
    x = F.relu(self.l1(x))
    
    y = self.l2(x)
    
    return x, y


net = CNN()
print(net)

CNN(
  (c1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (p1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (c2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (p2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (c3): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (p3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (l1): Linear(in_features=1024, out_features=1000, bias=True)
  (l2): Linear(in_features=1000, out_features=10, bias=True)
)


In [162]:
fake_input = torch.randn(2, 3, 32, 32, requires_grad=True)


inputs, outputs = net(fake_input)
outputs

tensor([[ 0.0510,  0.0280, -0.0313,  0.0550,  0.0383, -0.0170, -0.0233,  0.0270,
          0.0740, -0.0528],
        [ 0.0521,  0.0151, -0.0390,  0.0305,  0.0330,  0.0102, -0.0150,  0.0397,
          0.0871, -0.0337]], grad_fn=<AddmmBackward0>)

In [157]:
def Grad(inputs, outputs) :
    outputs_sum = outputs.sum()
    inputs.retain_grad()
    outputs_sum.backward()
    return torch.abs(inputs.grad)

In [158]:
Grad(inputs, outputs)

tensor([[0.0038, 0.0126, 0.0285,  ..., 0.0946, 0.0104, 0.0758],
        [0.0038, 0.0126, 0.0285,  ..., 0.0946, 0.0104, 0.0758]])

In [163]:
def GradxInput(inputs, outputs) :
    Grad_val = Grad(inputs, outputs)
    return Grad_val * inputs

In [164]:
GradxInput(inputs, outputs)

tensor([[0.0000, 0.0041, 0.0005,  ..., 0.0000, 0.0024, 0.0000],
        [0.0000, 0.0038, 0.0000,  ..., 0.0000, 0.0023, 0.0027]],
       grad_fn=<MulBackward0>)

In [165]:
def pal_loss(inputs, outputs, prior, attribution_method) :
    attribution_map = attribution_method(inputs, outputs)
    
    stand = torch.std(attribution_map)
    mean = torch.mean(attribution_map)
    
    res = ((attribution_map - mean) / stand) * prior
    return -res.sum()
    